# ROI Score

In [113]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

from keras.models import model_from_json
from keras.optimizers import SGD

import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

import ast
import sys  
from utils.ScoreGenerator import ScoreGenerator

In [114]:
import scipy.stats as sps
from scipy import stats
import statistics

In [115]:
fb = pd.concat([pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_facebook_1k-10k.tsv', delimiter='\t'),
                pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_facebook_10kplus.tsv', delimiter='\t')])

twitter = pd.concat([pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_twitter_1k-10k.tsv', delimiter='\t'),
                pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_twitter_10kplus.tsv', delimiter='\t')])

insta = pd.concat([pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_instagram_1k-10k.tsv', delimiter='\t'),
                pd.read_csv('/home/jovyan/capstone_data/listenfirst/data/view_brand_rollup_instagram_10kplus.tsv', delimiter='\t')])

In [116]:
clusters = pd.read_csv('utils/clusters.csv')
clusters['flop'] = np.where(clusters['DomesticGross']<clusters['budget'], 1, 0)

In [117]:
fb = pd.merge(fb, clusters, left_on = 'movie_id', right_on = 'brand_ods_id' , how = 'left')

fb['data_for']= pd.to_datetime(fb['data_for']) 
fb['released_on'] = pd.to_datetime(fb['released_on'])
fb['days_after_release'] = fb['data_for'] - fb['released_on']

In [118]:
## drop flop == NaN for now
fb = fb[fb['flop'].notna()]

## Non-parametric statistical tests for Facebook, 30 days before release

In [119]:
fb_30 = fb[fb['days_after_release'] == '-30 days']

In [120]:
fb_30['score'] = ((fb_30['total_post_interactions'] - fb_30['total_post_comments']) - \
               (0.5 * fb_30['total_post_sad_count'] + 0.5 * fb_30['total_post_angry_count'])) / \
               fb_30['total_post']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [121]:
fb_30_success = fb_30[fb_30['flop'] == 0]
fb_30_flop = fb_30[fb_30['flop'] == 1]

In [123]:
#fb_30_flop['score'].isna().sum()

1

In [124]:
#fb_30_flop[fb_30_flop['score'].isna()]

,movie_id,data_for,likes_c,likes,talking_about_c,talking_about,engagement_rate_c,engagement_rate,total_post_c,total_post,...,inflated_budget,budget_cat,inflated_boxoffice,year_q_bin,rank,production_company_bin,cluster_id,flop,days_after_release,score
2239516,60789,2012-12-27,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,-0.597882,MediumBudget,-0.679058,1.0,246.0,Top100+,5.0,1.0,-30 days,NaN


In [152]:
fb_30_success = fb_30_success.dropna()
fb_30_flop = fb_30_flop.dropna()

In [134]:
#fb_30_flop = fb_30_flop[fb_30_flop.movie_id != 60789]

In [135]:
#fb_30_flop['score'].isna().sum()

0

### Two sample ks test to compare distributions
#### Null hypothesis (H0): both samples come from the same distribution

In [136]:
KS_30 = sps.ks_2samp(fb_30_success['score'], fb_30_flop['score'])
KS_30

Ks_2sampResult(statistic=0.16888045540796964, pvalue=0.003529303383096538)

#### p-value = 0.004 < alpha = 0.05. Reject the null hypothesis and conclude that the two score distributions are different.

In [138]:
#####################################################################################################

### Kruskal Wallis Test to compare medians in two distributions
#### Null hypothesis (H0): population medians are equal.

In [139]:
## Calculate medians
Median_success_30 = statistics.median(fb_30_success['score'])
Median_success_30

5968.3628163074045

In [140]:
## Calculate medians
Median_flop_30 = statistics.median(fb_30_flop['score'])
Median_flop_30

3985.2727272727275

In [141]:
KW_30 = stats.kruskal(fb_30_success['score'], fb_30_flop['score'])
KW_30

KruskalResult(statistic=9.498658419171534, pvalue=0.002056221866569519)

#### p-value = 0.002 < alpha = 0.05. Reject the null hypothesis and conclude that score medians are different.

In [132]:
#################################################################################################################

### Mann Whitney U test to compare means in two distributions

In [133]:
MWU_30 = stats.mannwhitneyu(fb_30_success['score'], fb_30_flop['score'])
MWU_30

MannwhitneyuResult(statistic=23962.0, pvalue=0.0015813214793131943)

#### p-value = 0.0016 < alpha = 0.05. Reject the null hypothesis and conclude that score means are different.

## Non-parametric statistical tests for Facebook, 60 days before release

In [142]:
fb_60 = fb[fb['days_after_release'] == '-60 days']

In [143]:
fb_60['score'] = ((fb_60['total_post_interactions'] - fb_60['total_post_comments']) - \
               (0.5 * fb_60['total_post_sad_count'] + 0.5 * fb_60['total_post_angry_count'])) / \
               fb_60['total_post']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [144]:
fb_60_success = fb_60[fb_60['flop'] == 0]
fb_60_flop = fb_60[fb_60['flop'] == 1]

In [149]:
fb_60_success = fb_60_success.dropna()
fb_60_flop = fb_60_flop.dropna()

In [150]:
fb_60_success['score'].isna().sum()

0

In [151]:
fb_60_flop['score'].isna().sum()

0

### Two sample ks test to compare distributions
#### Null hypothesis (H0): both samples come from the same distribution

In [153]:
KS_60 = sps.ks_2samp(fb_60_success['score'], fb_60_flop['score'])
KS_60

Ks_2sampResult(statistic=0.21640706126687434, pvalue=0.0043058387232416395)

#### p-value = 0.004 < alpha = 0.05. Reject the null hypothesis and conclude that the two score distributions are different.

In [138]:
#####################################################################################################

### Kruskal Wallis Test to compare medians in two distributions
#### Null hypothesis (H0): population medians are equal.

In [154]:
## Calculate medians
Median_success_60 = statistics.median(fb_60_success['score'])
Median_success_60

8742.823475895053

In [155]:
## Calculate medians
Median_flop_60 = statistics.median(fb_60_flop['score'])
Median_flop_60

6217.512459150326

In [156]:
KW_60 = stats.kruskal(fb_60_success['score'], fb_60_flop['score'])
KW_60

KruskalResult(statistic=7.727950564322555, pvalue=0.005437243123927148)

#### p-value = 0.005 < alpha = 0.05. Reject the null hypothesis and conclude that score medians are different.

In [132]:
#################################################################################################################

### Mann Whitney U test to compare means in two distributions

In [157]:
MWU_60 = stats.mannwhitneyu(fb_60_success['score'], fb_60_flop['score'])
MWU_60

MannwhitneyuResult(statistic=7685.0, pvalue=0.002724610101378054)

#### p-value = 0.0027 < alpha = 0.05. Reject the null hypothesis and conclude that score means are different.

In [76]:
twitter = pd.merge(twitter, clusters, left_on = 'movie_id', right_on = 'brand_ods_id' , how = 'left')

twitter['data_for']= pd.to_datetime(twitter['data_for']) 
twitter['released_on'] = pd.to_datetime(twitter['released_on'])
twitter['days_after_release'] = twitter['data_for'] - twitter['released_on']

In [79]:
## drop flop == NaN for now
twitter = twitter[twitter['flop'].notna()]

In [82]:
twitter_30 = twitter[twitter['days_after_release'] == '-30 days']

In [77]:
insta = pd.merge(insta, clusters, left_on = 'movie_id', right_on = 'brand_ods_id' , how = 'left')

insta['data_for']= pd.to_datetime(insta['data_for']) 
insta['released_on'] = pd.to_datetime(insta['released_on'])
insta['days_after_release'] = insta['data_for'] - insta['released_on']

In [80]:
## drop flop == NaN for now
insta = insta[insta['flop'].notna()]

In [83]:
insta_30 = insta[insta['days_after_release'] == '-30 days']